In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from ordered_set import OrderedSet

## Load the data

In [4]:
data_path = './Group_Data/Group_7.xlsx'
flight_data = pd.read_excel(data_path, sheet_name='Flight', parse_dates=['Departure', 'Arrival'])
num_flight = flight_data.shape[0]
flight_data.shape, flight_data.columns

((232, 10),
 Index(['Flight Number', 'ORG', 'DEST', 'Departure', 'Arrival', 'A330', 'A340',
        'B737', 'B738', 'BUS'],
       dtype='object'))

In [5]:
itinerary_data = pd.read_excel(data_path, sheet_name='Itinerary')
num_itinerary = itinerary_data.shape[0]
itinerary_data.shape, itinerary_data.columns

((780, 8),
 Index(['Itin No.', 'Origin', 'Destination', 'Demand', 'Fare', 'Stops', 'Leg 1',
        'Leg 2'],
       dtype='object'))

In [6]:
recaptureRate_data = pd.read_excel(data_path, sheet_name='Recapture Rate')
num_recaptureRate = recaptureRate_data.shape[0]
recaptureRate_data.shape, recaptureRate_data.columns

((327, 3),
 Index(['From Itinerary', 'To Itinerary', 'Recapture Rate'], dtype='object'))

In [7]:
aircraft_data = pd.read_excel(data_path, sheet_name='Aircraft')
num_ac_type = aircraft_data.shape[0] - 1
num_type = aircraft_data.shape[0]
aircraft_data.shape, aircraft_data.columns

((5, 4), Index(['Type', 'Units', 'Seats', 'TAT'], dtype='object'))

## Preprocess the data into Python objects

### Create `Flight` objects

In [8]:
# define the class of Flight
class Flight:
    '''
    Define the class of flight:
    refers to your airline’s daily flight schedule, which
    contains, for each flight in the schedule, the flight number, the
    departure and arrival times, and operating costs for each aircraft
    type (in €).
    '''
    def __init__(self, 
        flight_number: str,
        origin: str,
        destin: str,
        departureTime, # datetime
        arrivalTime,   # datetime
        costs: dict
        ):
        self.flight_number = flight_number
        self.origin = origin
        self.destin = destin
        self.departureTime = departureTime
        self.arrivalTime = arrivalTime
        self.costs = costs
        
    def __repr__(self,):
        return self.flight_number + ':' + self.origin + '->' + self.destin
    
    def __str__(self,):
        return self.__repr__()
    
# construct the set of flights
flightSet = []
for flight in range(num_flight):
    flight_number = flight_data['Flight Number'][flight]
    origin = flight_data['ORG'][flight]
    destin = flight_data['DEST'][flight]
    departureTime = flight_data['Departure'][flight]
    arrivalTime = flight_data['Arrival'][flight]
    costs = {
            'A330':  flight_data['A330'][flight],
            'A340':  flight_data['A340'][flight],
            'B737':  flight_data['B737'][flight],
            'B738':  flight_data['B738'][flight],
            'BUS':  flight_data['BUS'][flight],
        }
    
    flightSet.append(
        Flight(
            flight_number = flight_number,
            origin = origin,
            destin = destin,
            departureTime = departureTime,
            arrivalTime = arrivalTime,
            costs = costs,
        )
    )
assert len(flightSet) == num_flight

### Create `Itinerary` objects

In [9]:
# define the class of Itinerary
class Itinerary:
    '''
    Define the class of itinerary:
    passenger itineraries, indicating the origin and destination, the demand and the fare (in
    €) for each itinerary. In addition, the flight or pair of flights used in each itinerary is provided
    '''
    def __init__(
        self,
        no: int,
        origin: str,
        destin: str,
        demand: int,
        fare: int,
        num_stops: int,
        leg1,
        leg2,
    ):
        self.no = no,
        self.origin = origin,
        self.destin = destin,
        self.demand = demand,
        self.fare = fare,
        self.num_stops = num_stops,
        self.leg1 = leg1,
        self.leg2 = leg2

# construct the set of itineraries
itinerarySet = []
for itinerary in range(num_itinerary):
    no = itinerary_data['Itin No.'][itinerary]
    origin = itinerary_data['Origin'][itinerary]
    destin = itinerary_data['Destination'][itinerary]
    demand = itinerary_data['Demand'][itinerary]
    fare = itinerary_data['Fare'][itinerary]
    num_stops = itinerary_data['Stops'][itinerary]
    leg1 = itinerary_data['Leg 1'][itinerary]
    leg2 = itinerary_data['Leg 2'][itinerary]
    
    itinerarySet.append(
        Itinerary(
            no=no,
            origin=origin,
            destin=destin,
            demand=demand,
            fare=fare,
            num_stops=num_stops,
            leg1=leg1,
            leg2=None if leg2 == 0 else leg2
        )
    )

# append the fictitious itinerary
itinerarySet.append(
    Itinerary(
        no = -1,
        origin=None,
        destin=None,
        demand=0,
        fare=0,
        num_stops=0,
        leg1=None,
        leg2=None
    )
)
    
assert len(itinerarySet) == num_itinerary + 1

### Create `Recapture Rate` set 

In [10]:
# construct the mappings between itinerary<From, To> and recapture rate
recaptureRateSet = {}

for idx in range(num_recaptureRate):
    # loop through all the recapture rates
    fromIti = recaptureRate_data['From Itinerary'].iloc[idx]
    toIti = recaptureRate_data['To Itinerary'].iloc[idx]
    recaptureRateSet[fromIti, toIti] = recaptureRate_data['Recapture Rate'].iloc[idx]

assert len(recaptureRateSet) == num_recaptureRate

### Read `aircraft` parameters

In [11]:
# retrieve the infos about aircraft types, units, seats and TATs
aircraftNames = ['A330', 'A340', 'B737', 'B738', 'BUS']
units = aircraft_data['Units']
seats = aircraft_data['Seats']
TATs = aircraft_data['TAT']

### Create `Node` objects

In [50]:
class Node:
    '''
    Based on the time-space network, 
    a node is the position that aircraft departs or arrives at airport i in the time t.
    '''
    def __init__(
        self,
        airport,
        timestamp,
    ):
        self.airport = airport
        self.timestamp = timestamp
        self.num_airports_in = 1
        self.num_airports_out = 1
        self.nodeNo = None
        
    def setNodeNo(self, no):
        self.nodeNo = no
        
    def __repr__(self):
        return self.airport+str(self.timestamp)+' '+str(self.nodeNo)
    
    def __str__(self):
        return self.__repr__()

    def __key(self):
        return (self.airport, self.timestamp)

    def __hash__(self):
        return hash(self.__key())

    def __eq__(self, other):
        if isinstance(other, Node):
            return self.__key() == other.__key()
        return NotImplemented        

# construct the set of Nodes
nodeSet = OrderedSet()
no = 0
for flight in flightSet:
    # create the node where an flight departs
    node = Node(airport=flight.origin, timestamp=flight.departureTime)
    if node not in nodeSet:
        node.setNodeNo(no)
        no += 1
        nodeSet.add(node)
    else:
        loc = nodeSet.get_loc(node)
        nodeSet[loc].num_airports_out += 1
    # create the node where an flight arrives
    node = Node(airport=flight.destin, timestamp=flight.arrivalTime)
    if node not in nodeSet:
        node.setNodeNo(no)
        no += 1
        nodeSet.add(node)
    else:
        loc = nodeSet.get_loc(node)
        nodeSet[loc].num_airports_in += 1
len(nodeSet)

422

### Create `Ground` objects

In [60]:
class Ground:
    def __init__(
        self,
        airport,
        begin_timestamp,
        end_timestamp,
        begin_node,
        end_node,
        isOvernight = False
    ):
        self.airport = airport
        self.begin_timestamp = begin_timestamp
        self.end_timestamp = end_timestamp
        self.beginNode = None
        self.endNode = begin_node
        self.groundNo = end_node
        self.isOvernight = isOvernight
        
    def setGroundNo(self, no):
        self.groundNo = no

In [72]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x.timestamp - pivot.timestamp))

def earlyest(items, pivot):
    return max(items, key=lambda x: x.timestamp - pivot.timestamp)

# construct the set of Grounds
groundSet = OrderedSet()
no = 0

for node_1 in nodeSet:
    afterNodes = []
    potentialNodes = []
    for node_2 in nodeSet:
        if node_1.nodeNo == node_2.nodeNo:
            break 
        elif node_1.airport == node_2.airport:
            potentialNodes.append(node_2) # append all the nodes in the same airport into the potentialNode list.
            if node_2.timestamp > node_1.timestamp:
                afterNodes.append(node_2)     # append all the nodes that are in the same airport and after the current node.
        
#     # search for the closest datatime of the current node to create a ground object
#     if len(afterNodes) > 0:
#         node = nearest(afterNodes, pivot=node_1)
#         ground = Ground(node_1.airport, node_1.timestamp, node_2.timestamp, node_1, node_2, False)
#         ground.setGroundNo(no)
#         no += 1
        
#     # indicates that this is an overnight ground arc
#     if len(afterNodes) == 0:
#         node = earlyest(potentialNodes, pivot=node_1) if (len(potentialNodes) > 0) else []
#         print(node, node_1, potentialNodes)
#         break
    # if potentialNodes.__len__() == 0:
    print(potentialNodes)

[]
[]
[LIME2023-01-03 09:25:00 1]
[LIMC2023-01-03 08:00:00 0]
[LIME2023-01-03 09:25:00 1, LIME2023-01-03 08:00:00 2]
[]
[KJFK2023-01-03 11:10:00 5]
[LIME2023-01-03 09:25:00 1, LIME2023-01-03 08:00:00 2, LIME2023-01-03 23:55:00 4]
[]
[]
[LIME2023-01-03 09:25:00 1, LIME2023-01-03 08:00:00 2, LIME2023-01-03 23:55:00 4, LIME2023-01-03 04:20:00 7]
[]
[ZBAA2023-01-03 10:35:00 11]
[LIME2023-01-03 09:25:00 1, LIME2023-01-03 08:00:00 2, LIME2023-01-03 23:55:00 4, LIME2023-01-03 04:20:00 7, LIME2023-01-03 22:55:00 10]
[LIMC2023-01-03 08:00:00 0, LIMC2023-01-03 09:25:00 3]
[LIME2023-01-03 09:25:00 1, LIME2023-01-03 08:00:00 2, LIME2023-01-03 23:55:00 4, LIME2023-01-03 04:20:00 7, LIME2023-01-03 22:55:00 10, LIME2023-01-03 03:00:00 13]
[LIMC2023-01-03 08:00:00 0, LIMC2023-01-03 09:25:00 3, LIMC2023-01-03 10:00:00 14]
[]
[FACT2023-01-03 09:15:00 17]
[LIMC2023-01-03 08:00:00 0, LIMC2023-01-03 09:25:00 3, LIMC2023-01-03 10:00:00 14, LIMC2023-01-03 07:00:00 16]
[LIMC2023-01-03 08:00:00 0, LIMC2023-01-

## Column Generation Algorithm (CG)

### Solve the initial RMP

#### Decision variables

In [26]:
# In the initial RMP, a 'fictitious' itinerary is considered as the buffer for spillage.

model = gp.Model('RMP')

# declare the decision variables
# f_i^k: binary variable if flight arc i is assigned to aircraft type k, otherwise 0 
# (assumed continous in RMF setting)
f = model.addVars(num_flight, num_ac_type,)
assert len(f) == num_flight * num_ac_type
# y_a^k: integer variable indicating the number of aircrafts of type k on the ground arc a 
# (assumed continous in RMF setting)
# y = model.addVars(
#     
# )

# t_p^r: the number of passengers that would like to travel on itinerary p but are reallocated by the airline to itinerary r
num_initial = num_itinerary

t = model.addVars(num_initial,1)
assert len(t) == num_initial

### Solve the relaxed IFAM problem

### Solve the relaxed IFAM problem with integer DVs